In [1]:
'''
基于PointNet++的CNN模型框架，一些细节并不正确。
输入是点云和评分，输出是训练好的神经网络，可用于对新点云评分。
如果接受更大的算量，接下来还可以试一下体素模型（把输入端的点云采样改为体素，前者是从n个点里选m个，后者是把n个点扩展为N个，除此之外算法也要重写）。
'''

'\n基于PointNet++的CNN模型，这个是基础版。\n输入是点云和评分，输出是训练好的神经网络，可用于对新点云评分。\n如果接受更大的算量，接下来还可以试一下体素模型（把输入端的点云采样改为体素，前者是从n个点里选m个，后者是把n个点扩展为N个，除此之外算法也要重写）。\n'

In [2]:
'''
# 安装环境，和上次发的代码一样的操作方式
!pip install --upgrade tensorflow
'''

'\n# 安装环境，和上次发的代码一样的操作方式\n!pip install --upgrade tensorflow\n'

In [3]:
'''
# 更新环境，以解决numpy和tensorflow的兼容性问题
pip install --upgrade numpy tensorflow
'''

'\n# 更新环境，以解决numpy和tensorflow的兼容性问题\npip install --upgrade numpy tensorflow\n'

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

2024-07-02 23:16:24.045229: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-02 23:16:24.049545: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-02 23:16:24.063277: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 23:16:24.170915: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 23:16:24.170988: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-02 23:16:24.191362: I tensorflow/core/platform/cpu_feature_guard.cc:

In [5]:
def read_scores(file_path):
    # 读全部牙的序号和评分
    indices = []
    scores = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 2:
                try:
                    index = int(parts[0])
                    score = float(parts[1])
                    indices.append(index)
                    scores.append(score)
                except ValueError:
                    print(f"Skipping line due to value error: {line.strip()}")
            else:
                print(f"Skipping line due to incorrect format: {line.strip()}")
    return indices, scores

def read_obj_file(file_path):
    # 读单颗牙的3D模型
    vertices = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('v '):
                parts = line.strip().split()
                vertex = list(map(float, parts[1:4]))
                vertices.append(vertex)
    return np.array(vertices)

def sample_points(points, num_samples):
    # 采样
    # 由于每颗牙的3D模型顶点数不同（约2100+），不方便直接用作神经网络的输入，需要统一输入的格式
    # 如果某颗牙的顶点数大于采样点数，则采样不可重复，否则采样可重复
    # 为了和vertices做变量名上的区分，这个函数中使用了points这个变量名，但实际上就是vertices的数据
    # 此处使用的是点云模型，如果用体素模型则有其他的配套手段
    if len(points) >= num_samples:
        sampled_points = points[np.random.choice(len(points), num_samples, replace=False)]
    else:
        sampled_points = points[np.random.choice(len(points), num_samples, replace=True)]
    return sampled_points


In [6]:
# PointNet++模型

def pointnet_sa_module(xyz, npoint, mlp):
    '''
    Set Abstraction模块，用于提取点云的局部特征
    其输入为：
    xyz = 输入点云的3维坐标xzy
    npoint = 采样点数
    mlp = 多层感知机(Multilayer Perceptron)每层神经元数量的列表
    其输出为：
    采样后的点云new_xyz，形状为npoint*3
    提取到的特征new_points，形状为npoints*out_channel，也就是点数不变但维数变为多层感知机最后一层的节点数
    '''
    # 采样点云的前 npoint 个点，可以压缩数据量，也可以令每颗牙的数据尺寸一致化
    '''！！！这里有问题！！！'''
    new_xyz = xyz[:npoint, :]
    
    # 用1维卷积做特征提取
    # 1维卷积只能将同一点的不同特征做混合，不同点相对位置引起的特征是靠对点云的采样和分组实现的
    new_points = new_xyz
    for out_channel in mlp:
        new_points = layers.Conv1D(out_channel, 1, activation='relu')(new_points)
    return new_xyz, new_points

def build_pointnetplusplus(input_shape):
    '''
    PointNet++模型，需要调用SA模块
    输入为“单颗牙的3D模型在采样后的数据格式”，即num_samples*3
    输出为训练好的模型
    '''
    #输入层
    inputs = tf.keras.Input(shape=input_shape)
    l0_xyz = inputs
    
    # 三个采样和特征提取层，其中第三层不做采样。需调用SA模块
    l1_xyz, l1_points = pointnet_sa_module(l0_xyz, npoint=512, mlp=[64, 64, 128])
    l2_xyz, l2_points = pointnet_sa_module(l1_xyz, npoint=128, mlp=[128, 128, 256])
    l3_xyz, l3_points = pointnet_sa_module(l2_xyz, npoint=None, mlp=[256, 512, 1024])
    
    # 全局最大池化层，用于压缩数据量，也实现了数据扁平化
    x = layers.GlobalMaxPooling1D()(l3_points)
    
    # 全连接层和Dropout层，前者用于进行特征的整合和最终的输出，后者用于减少神经网络的过拟合
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    
    # 输出层
    outputs = layers.Dense(1)(x)
    
    # 构建模型
    model = models.Model(inputs=inputs, outputs=outputs)
    
    '''
    编译模型：
    Adam优化器用于加快训练
    均方误差Mean Squared Error作为损失函数
    平均绝对误差Mean Absolute Error作为评价指标
    '''
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    
    return model


In [7]:
# 读序号和评分
'''此处为text文件地址，应当改为：’文件夹路径 + scores.txt'''
score_file_path = 'scores.txt'
indices, scores = read_scores(score_file_path)

# 读3D模型，并完成采样
'''设置采样点数为1024，可修改'''
num_samples = 1024  
Vertices = []
for i in indices:
    obj_file_path = f'{i}.obj'  # 此处为obj文件地址，应当改为：f’ + 文件夹路径 + {i}.obj
    points = read_obj_file(obj_file_path)
    sampled_points = sample_points(points, num_samples)
    Vertices.append(sampled_points)
Vertices = np.array(Vertices)
Scores = np.array(scores)

# 划分训练集和测试集
'''此处用的4：1随机分配，以后可以改成k折'''
Vertices_train, Vertices_test, Scores_train, Scores_test = train_test_split(Vertices, Scores, test_size=0.2, random_state=42)

In [8]:
# 训练
model = build_pointnetplusplus((num_samples, 3))
history = model.fit(Vertices_train, Scores_train, epochs=5, batch_size=8, validation_data=(Vertices_test, Scores_test))

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - loss: 2330.0386 - mae: 42.7014 - val_loss: 481.8236 - val_mae: 19.5795
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - loss: 562.5247 - mae: 17.6014 - val_loss: 477.5894 - val_mae: 18.7642
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - loss: 415.3473 - mae: 16.4173 - val_loss: 441.5332 - val_mae: 19.1154
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - loss: 403.6625 - mae: 16.9036 - val_loss: 355.3603 - val_mae: 15.9048
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 24s 2s/step - loss: 309.9753 - mae: 13.1624 - val_loss: 287.1973 - val_mae: 13.6874


In [9]:
# 模型评价
loss, mae = model.evaluate(Vertices_test, Scores_test)
print("Test Loss:", loss)
print("Test MAE:", mae)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 287.1973 - mae: 13.6874
Test Loss: 287.19732666015625
Test MAE: 13.68735122680664


In [10]:
# 保存模型
model.save('saved_model.keras')

In [11]:
# 加载模型
loaded_model = tf.keras.models.load_model('saved_model.keras')

In [12]:
def evaluate(new_obj_file_path):
    # 读取并预处理新OBJ文件
    new_vertices = read_obj_file(new_obj_file_path)
    new_sampled_points = sample_points(new_vertices, num_samples)
    new_sampled_points = np.expand_dims(new_sampled_points, axis=0)  # 添加批次维度
    
    # 使用加载的模型进行预测
    predicted_score = loaded_model.predict(new_sampled_points)
    # print("Predicted Score:", predicted_score[0])
    
    return predicted_score

evaluate_result = {}
for i in range(150):
    new_obj_file_path = f'{i+1}.obj'  # 待评价对象的地址
    evaluate_result[i] = evaluate(new_obj_file_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━

In [13]:
evaluate_result

{0: array([[64.92832]], dtype=float32),
 1: array([[61.108406]], dtype=float32),
 2: array([[61.335632]], dtype=float32),
 3: array([[66.16812]], dtype=float32),
 4: array([[62.129997]], dtype=float32),
 5: array([[55.568108]], dtype=float32),
 6: array([[55.61471]], dtype=float32),
 7: array([[63.957825]], dtype=float32),
 8: array([[61.437004]], dtype=float32),
 9: array([[59.736763]], dtype=float32),
 10: array([[66.75143]], dtype=float32),
 11: array([[60.286205]], dtype=float32),
 12: array([[62.081223]], dtype=float32),
 13: array([[65.43191]], dtype=float32),
 14: array([[62.682987]], dtype=float32),
 15: array([[56.22727]], dtype=float32),
 16: array([[57.138054]], dtype=float32),
 17: array([[63.487835]], dtype=float32),
 18: array([[62.372684]], dtype=float32),
 19: array([[59.118942]], dtype=float32),
 20: array([[66.669495]], dtype=float32),
 21: array([[60.787827]], dtype=float32),
 22: array([[60.803593]], dtype=float32),
 23: array([[64.47334]], dtype=float32),
 24: arra